Load and Feature Engineering

In [1]:
import pandas as pd 

df=pd.read_csv('cleaned_network_data.csv')

In [6]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 42 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   timestamp                         9000 non-null   object 
 1   tower_id                          9000 non-null   object 
 2   latency_sec                       9000 non-null   float64
 3   bandwidth                         9000 non-null   object 
 4   dropped_calls                     9000 non-null   int64  
 5   total_calls                       9000 non-null   int64  
 6   uptime_percent                    9000 non-null   float64
 7   network_type                      9000 non-null   object 
 8   operator                          9000 non-null   object 
 9   users_connected                   9000 non-null   int64  
 10  download_speed_mbps               9000 non-null   float64
 11  signal_strength_dbm               9000 non-null   float64
 12  tower_

(9000, 42)

Drop unrequierd rows

In [7]:
performance_metrics = [
        "latency_sec", "dropped_calls",
        "total_calls", "call_drop_rate", "uptime_percent",
        "users_connected", "signal_strength.RSSI",
        "signal_strength.RSRP", "signal_strength.SINR",
        "voip_metrics.jitter_ms", "voip_metrics.packet_loss_percent"
    ]

df=df[performance_metrics]

In [9]:
df.tail()

,latency_sec,dropped_calls,total_calls,call_drop_rate,uptime_percent,users_connected,signal_strength.RSSI,signal_strength.RSRP,signal_strength.SINR,voip_metrics.jitter_ms,voip_metrics.packet_loss_percent
8995,0.315,8,150,5.333333,99.08,248,-75.99,-83.66,13.04,31.88,1.34
8996,0.336,3,138,2.173913,98.96,376,-65.03,-87.21,28.68,30.59,1.53
8997,0.792,10,171,5.847953,99.03,247,-61.80,-106.96,-6.81,1.35,1.00
8998,0.746,1,186,0.537634,99.97,101,-80.17,-131.99,23.03,48.12,3.88
8999,0.163,2,79,2.531646,95.78,164,-73.75,-112.53,10.91,17.34,4.98


In [12]:
df.isna().sum()

latency_sec                         0
dropped_calls                       0
total_calls                         0
call_drop_rate                      0
uptime_percent                      0
users_connected                     0
signal_strength.RSSI                0
signal_strength.RSRP                0
signal_strength.SINR                0
voip_metrics.jitter_ms              0
voip_metrics.packet_loss_percent    0
dtype: int64

In [ ]:
def evaluate_row(row):
    # Call Drop Rate
    if row['call_drop_rate'] >= 2:
        return 1
    # Latency
    if row['latency_sec'] >= 0.15:
        return 1
    # Signal strength
    if row['signal_strength.RSSI'] <= -90:
        return 1
    if row['signal_strength.RSRP'] <= -100:
        return 1
    if row['signal_strength.SINR'] <= 20:
        return 1
    # VoIP metrics
    if row['voip_metrics.jitter_ms'] >= 30:
        return 1
    if row['voip_metrics.packet_loss_percent'] >= 1:
        return 1
    # Uptime
    if row['uptime_percent'] <= 99.9:
        return 1

    return 0